In [2]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin

In [57]:
def get_all_links(url, depth=0, visited={}):
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  links = soup.find_all('a')
  links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
  links = [link for link in links if link.startswith(url) or link.startswith('/')]
  links = [urljoin(url, link) for link in links if link]


  # Recursively crawl the links at the next depth level
  if depth < 1: #3
    new_links = []
    for link in links:
      # Increment the visit count for the link
      if link in visited:
        visited[link] += 1
      else:
        visited[link] = 1
      # Get the newly-crawled links and add them to the list
        new_links.extend(get_all_links(link, depth=depth+1, visited=visited))
      # Add the newly-crawled links to the original list
    links.extend(new_links)

  return visited

base_url = 'https://www.thairath.co.th'
website_dict = get_all_links(base_url, depth=0, visited={})
print(website_dict)



{'https://www.thairath.co.th/home': 1, 'https://www.thairath.co.th/tv/live': 2, 'https://www.thairath.co.th/news/politic/2611696': 4, 'https://www.thairath.co.th/news/society/2611708': 4, 'https://www.thairath.co.th/news/foreign/2611565': 4, 'https://www.thairath.co.th/lottery/news/2611380': 4, 'https://www.thairath.co.th/news/local/2611523': 4, 'https://www.thairath.co.th/scoop/interview/2611308': 4, 'https://www.thairath.co.th/spotlight': 2, 'https://www.thairath.co.th/spotlight/covid-19-vaccine': 6, 'https://www.thairath.co.th/spotlight/bualuang-printing': 4, 'https://www.thairath.co.th/spotlight/bblfinancialtree/': 4, 'https://www.thairath.co.th/spotlight/onlinelearning': 4, 'https://www.thairath.co.th/column': 2, 'https://www.thairath.co.th/news/auto/news/2611385': 2, 'https://www.thairath.co.th/news/local/2610793': 1, 'https://www.thairath.co.th/column/newspaper/page1scoop': 1, 'https://www.thairath.co.th/news/politic/2611423': 1, 'https://www.thairath.co.th/column/newspaper/wikr

In [61]:
def crawl(url, depth,visited = set()):
    if depth < 3 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [link for link in links if link.startswith(url) or link.startswith('/')]
        links = [urljoin(url, link) for link in links if link]
        for link in links:
            crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://www.thairath.co.th'
website_dict = crawl(base_url, 0, visited=set())
print(website_dict)


KeyboardInterrupt: 

In [59]:
a={'https://www.thairath.co.th/news/politic/2611696', 'https://www.thairath.co.th/spotlight/bualuang-printing', 'https://www.thairath.co.th/lifestyle/tech/2611086', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851', 'https://www.thairath.co.th/scoop/world', 'https://www.thairath.co.th/multimedia/gallery', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/704356', 'https://www.thairath.co.th/ads/thairathOnline', 'https://www.thairath.co.th/spotlight/covid-19-vaccine', 'https://www.thairath.co.th/news/society/2611708', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860', 'https://www.thairath.co.th/tv/program/thairath_news_show', 'https://www.thairath.co.th/news/foreign/2610783', 'https://www.thairath.co.th/news/politic/2611423', 'https://www.thairath.co.th/column/newspaper/wikroh', 'https://www.thairath.co.th/', 'https://www.thairath.co.th/lifestyle/fashion/2611583', 'https://www.thairath.co.th/scoop/culture/2607981', 'https://www.thairath.co.th/lifestyle', 'https://www.thairath.co.th/sport/thaifootball/2611678', 'https://www.thairath.co.th/spotlight', 'https://www.thairath.co.th/news/local/2611523', 'https://www.thairath.co.th/lottery', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714895', 'https://www.thairath.co.th/tv/program/rueng_rop_khop_sanam/714893', 'https://www.thairath.co.th/business/investment/gold/history', 'https://www.thairath.co.th/scoop', 'https://www.thairath.co.th/tv/program/buntoeng_thairath', 'https://www.thairath.co.th/tv', 'https://www.thairath.co.th/tv/program', 'https://www.thairath.co.th/podcast', 'https://www.thairath.co.th/scoop/theissue/2611330', 'https://www.thairath.co.th/tv/live', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34308', 'https://www.thairath.co.th/news/local/2610793', 'https://www.thairath.co.th/news/local/2594348', 'https://www.thairath.co.th/tv/program/rueng_rop_khop_sanam/714891', 'https://www.thairath.co.th/scoop/world/2610365', 'https://www.thairath.co.th/tv/watch', 'https://www.thairath.co.th/podcast/program/Howtolove/episode140', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang', 'https://www.thairath.co.th/horoscope/dailyhoro/2608620', 'https://www.thairath.co.th/column/newspaper/gladai', 'https://www.thairath.co.th/news/politic/2610866', 'https://www.thairath.co.th/scoop/theissue/2610376', 'https://www.thairath.co.th/scoop/interview', 'https://www.thairath.co.th/member/profile/following', 'https://www.thairath.co.th/lifestyle/life/2605256', 'https://www.thairath.co.th/spotlight/bblfinancialtree/', 'https://www.thairath.co.th/lottery/news/2611380', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218', 'https://www.thairath.co.th/lifestyle/life/2611281', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225', 'https://www.thairath.co.th/tv/program/talon_khao', 'https://www.thairath.co.th/scoop/theissue/2611335', 'https://www.thairath.co.th/refresh', 'https://www.thairath.co.th/business/investment/gold/topic', 'https://www.thairath.co.th/tags/พ.พาทินี', 'https://www.thairath.co.th/tv/watch/news', 'https://www.thairath.co.th/column/newspaper/thai_remark', 'https://www.thairath.co.th/horoscope/horoscope-seer', 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew', 'https://www.thairath.co.th/tv/program/khao_tieng_thairath', 'https://www.thairath.co.th/tv/program/khoa_yen_thairath', 'https://www.thairath.co.th/business/market', 'https://www.thairath.co.th/tv/program/khao_sai_khai', 'https://www.thairath.co.th/business/investment/gold', 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033', 'https://www.thairath.co.th/business/market/2608195', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330', 'https://www.thairath.co.th/scoop/theissue', 'https://www.thairath.co.th/tv/program/khao_tieng_thairath/714883', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101', 'https://www.thairath.co.th/podcast/program/extratime/episode62', 'https://www.thairath.co.th/campaign/vote', 'https://www.thairath.co.th/scoop/interview/2611308', 'https://www.thairath.co.th/column/newspaper/page1scoop', 'https://www.thairath.co.th/lifestyle/life/2610961', 'https://www.thairath.co.th/lifestyle/life', 'https://www.thairath.co.th/scoop/interview/2608225', 'https://www.thairath.co.th/news/foreign/2611565', 'https://www.thairath.co.th/spotlight/onlinelearning', 'https://www.thairath.co.th/column', 'https://www.thairath.co.th/video', 'https://www.thairath.co.th/tv/program/rueng_rop_khop_sanam', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265', 'https://www.thairath.co.th/home', 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208', 'https://www.thairath.co.th/lifestyle/fashion', 'https://www.thairath.co.th', 'https://www.thairath.co.th/lifestyle/tech'}
b={'https://www.thairath.co.th/home': 12, 'https://www.thairath.co.th/news/politic/2611696': 8, 'https://www.thairath.co.th/news/society/2611708': 8, 'https://www.thairath.co.th/news/foreign/2611565': 8, 'https://www.thairath.co.th/lottery/news/2611380': 8, 'https://www.thairath.co.th/news/local/2611523': 8, 'https://www.thairath.co.th/scoop/interview/2611308': 8, 'https://www.thairath.co.th/spotlight': 4, 'https://www.thairath.co.th/column': 4, 'https://www.thairath.co.th/news/local/2610793': 4, 'https://www.thairath.co.th/column/newspaper/page1scoop': 2, 'https://www.thairath.co.th/news/politic/2611423': 2, 'https://www.thairath.co.th/column/newspaper/wikroh': 2, 'https://www.thairath.co.th/sport/thaifootball/2611678': 2, 'https://www.thairath.co.th/news/politic/2610866': 2, 'https://www.thairath.co.th/column/newspaper/gladai': 2, 'https://www.thairath.co.th/news/foreign/2610783': 2, 'https://www.thairath.co.th/column/newspaper/thai_remark': 2, 'https://www.thairath.co.th/member/profile/following': 4, 'https://www.thairath.co.th/scoop': 4, 'https://www.thairath.co.th/scoop/theissue/2611335': 4, 'https://www.thairath.co.th/scoop/theissue': 6, 'https://www.thairath.co.th/scoop/theissue/2611330': 4, 'https://www.thairath.co.th/scoop/world/2610365': 4, 'https://www.thairath.co.th/scoop/world': 2, 'https://www.thairath.co.th/scoop/theissue/2610376': 4, 'https://www.thairath.co.th/scoop/interview/2608225': 4, 'https://www.thairath.co.th/scoop/interview': 2, 'https://www.thairath.co.th/refresh': 6, 'https://www.thairath.co.th/lifestyle/life/2611281': 8, 'https://www.thairath.co.th/lifestyle/life/2610961': 8, 'https://www.thairath.co.th/scoop/culture/2607981': 4, 'https://www.thairath.co.th/video': 4, 'https://www.thairath.co.th/podcast': 4, 'https://www.thairath.co.th/podcast/program/Howtolove/episode140': 4, 'https://www.thairath.co.th/podcast/program/extratime/episode62': 4, 'https://www.thairath.co.th/lifestyle': 4, 'https://www.thairath.co.th/lifestyle/life/2605256': 4, 'https://www.thairath.co.th/lifestyle/life': 6, 'https://www.thairath.co.th/business/market/2608195': 4, 'https://www.thairath.co.th/business/market': 2, 'https://www.thairath.co.th/horoscope/dailyhoro/2608620': 4, 'https://www.thairath.co.th/tags/พ.พาทินี': 2, 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033': 4, 'https://www.thairath.co.th/horoscope/horoscope-seer': 2, 'https://www.thairath.co.th/lifestyle/fashion/2611583': 4, 'https://www.thairath.co.th/lifestyle/fashion': 2, 'https://www.thairath.co.th/lifestyle/tech/2611086': 4, 'https://www.thairath.co.th/lifestyle/tech': 2, 'https://www.thairath.co.th/tv': 17, 'https://www.thairath.co.th/tv/program': 17, 'https://www.thairath.co.th/tv/live': 18, 'https://www.thairath.co.th/tv/watch/news': 14, 'https://www.thairath.co.th/tv/program/rueng_rop_khop_sanam/714893': 8, 'https://www.thairath.co.th/tv/program/rueng_rop_khop_sanam': 4, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath/714891': 8, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath': 9, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath/714883': 4, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath/714880': 4, 'https://www.thairath.co.th/news/local/2594348': 4, 'https://www.thairath.co.th/': 12, 'https://www.thairath.co.th/tv/watch': 19, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101': 4, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang': 8, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265': 4, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330': 4, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/704356': 4, 'https://www.thairath.co.th/lottery': 6, 'https://www.thairath.co.th/business/investment/gold/history': 4, 'https://www.thairath.co.th/business/investment/gold': 2, 'https://www.thairath.co.th/business/investment/gold/topic': 2, 'https://www.thairath.co.th/multimedia/gallery': 4, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218': 12, 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208': 12, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860': 12, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851': 12, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34308': 8, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225': 8, 'https://www.thairath.co.th/spotlight/covid-19-vaccine': 6, 'https://www.thairath.co.th/news': 1, 'https://www.thairath.co.th/spotlight/bualuang-printing': 4, 'https://www.thairath.co.th/spotlight/bblfinancialtree/': 4, 'https://www.thairath.co.th/spotlight/onlinelearning': 4, 'https://www.thairath.co.th/campaign/vote': 2, 'https://www.thairath.co.th/job': 2, 'https://www.thairath.co.th/aboutus': 2, 'https://www.thairath.co.th/contact_th': 17, 'https://www.thairath.co.th/ads': 4, 'https://www.thairath.co.th/campaign/vote/rule': 2, 'https://www.thairath.co.th/campaign/vote/prize': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew': 1, 'https://www.thairath.co.th/tv/schedule': 15, 'https://www.thairath.co.th/tv/anchor': 10, 'https://www.thairath.co.th/tv/watch/variety': 10, 'https://www.thairath.co.th/tv/watch/sport': 10, 'https://www.thairath.co.th/ads/thairathTV': 17, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714751': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714750': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714745': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714743': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714740': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714738': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714735': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714733': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714726': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714723': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714721': 2, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714716': 2, 'https://www.thairath.co.th/tv/program/talon_khao': 1, 'https://www.thairath.co.th/tv/program/talon_khao/714801': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714796': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714791': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714788': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714785': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714780': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714778': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714773': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714771': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714768': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714765': 2, 'https://www.thairath.co.th/tv/program/talon_khao/714761': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath': 1, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714530': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714533': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714528': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714525': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714521': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714520': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714518': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714515': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714516': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714513': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714508': 2, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath/714511': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai': 1, 'https://www.thairath.co.th/tv/program/khao_sai_khai/714576': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/714106': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/713123': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/712803': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/711808': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/711283': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/710348': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/709933': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/709433': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/708951': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/708421': 2, 'https://www.thairath.co.th/tv/program/khao_sai_khai/707511': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show': 1, 'https://www.thairath.co.th/tv/program/thairath_news_show/714651': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714648': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714643': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714638': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714640': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714635': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714630': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714625': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714620': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714616': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714615': 2, 'https://www.thairath.co.th/tv/program/thairath_news_show/714611': 2, 'https://www.thairath.co.th/ads/thairathOnline': 3, 'https://www.thairath.co.th/ads/newspaper': 2}
c={'https://www.thairath.co.th/home': 1, 'https://www.thairath.co.th/tv/live': 2, 'https://www.thairath.co.th/news/politic/2611696': 4, 'https://www.thairath.co.th/news/society/2611708': 4, 'https://www.thairath.co.th/news/foreign/2611565': 4, 'https://www.thairath.co.th/lottery/news/2611380': 4, 'https://www.thairath.co.th/news/local/2611523': 4, 'https://www.thairath.co.th/scoop/interview/2611308': 4, 'https://www.thairath.co.th/spotlight': 2, 'https://www.thairath.co.th/spotlight/covid-19-vaccine': 6, 'https://www.thairath.co.th/spotlight/bualuang-printing': 4, 'https://www.thairath.co.th/spotlight/bblfinancialtree/': 4, 'https://www.thairath.co.th/spotlight/onlinelearning': 4, 'https://www.thairath.co.th/column': 2, 'https://www.thairath.co.th/news/auto/news/2611385': 2, 'https://www.thairath.co.th/news/local/2610793': 1, 'https://www.thairath.co.th/column/newspaper/page1scoop': 1, 'https://www.thairath.co.th/news/politic/2611423': 1, 'https://www.thairath.co.th/column/newspaper/wikroh': 1, 'https://www.thairath.co.th/sport/thaifootball/2611678': 1, 'https://www.thairath.co.th/news/politic/2610866': 1, 'https://www.thairath.co.th/column/newspaper/gladai': 1, 'https://www.thairath.co.th/member/profile/following': 2, 'https://www.thairath.co.th/scoop': 2, 'https://www.thairath.co.th/scoop/theissue/2611335': 2, 'https://www.thairath.co.th/scoop/theissue': 3, 'https://www.thairath.co.th/scoop/theissue/2611330': 2, 'https://www.thairath.co.th/scoop/world/2610365': 2, 'https://www.thairath.co.th/scoop/world': 1, 'https://www.thairath.co.th/scoop/theissue/2610376': 2, 'https://www.thairath.co.th/scoop/interview/2608225': 2, 'https://www.thairath.co.th/scoop/interview': 1, 'https://www.thairath.co.th/campaign/vote': 1, 'https://www.thairath.co.th/refresh': 3, 'https://www.thairath.co.th/lifestyle/life/2611281': 4, 'https://www.thairath.co.th/lifestyle/life/2610961': 4, 'https://www.thairath.co.th/scoop/culture/2607981': 2, 'https://www.thairath.co.th/video': 2, 'https://www.thairath.co.th/podcast': 2, 'https://www.thairath.co.th/podcast/program/Howtolove/episode140': 2, 'https://www.thairath.co.th/podcast/program/extratime/episode62': 2, 'https://www.thairath.co.th/lifestyle': 2, 'https://www.thairath.co.th/lifestyle/life/2605256': 2, 'https://www.thairath.co.th/lifestyle/life': 3, 'https://www.thairath.co.th/business/market/2608195': 2, 'https://www.thairath.co.th/business/market': 1, 'https://www.thairath.co.th/horoscope/dailyhoro/2608620': 2, 'https://www.thairath.co.th/tags/พ.พาทินี': 1, 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033': 2, 'https://www.thairath.co.th/horoscope/horoscope-seer': 1, 'https://www.thairath.co.th/lifestyle/fashion/2611583': 2, 'https://www.thairath.co.th/lifestyle/fashion': 1, 'https://www.thairath.co.th/lifestyle/tech/2611086': 2, 'https://www.thairath.co.th/lifestyle/tech': 1, 'https://www.thairath.co.th/tv': 1, 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew': 1, 'https://www.thairath.co.th/tv/program/talon_khao': 1, 'https://www.thairath.co.th/tv/program/khao_tieng_thairath': 1, 'https://www.thairath.co.th/tv/program/khoa_yen_thairath': 1, 'https://www.thairath.co.th/tv/program/khao_sai_khai': 1, 'https://www.thairath.co.th/tv/program/thairath_news_show': 1, 'https://www.thairath.co.th/tv/program': 1, 'https://www.thairath.co.th/tv/watch/news': 2, 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714935': 4, 'https://www.thairath.co.th/tv/program/buntoeng_thairath': 6, 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714921': 4, 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714913': 2, 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714915': 2, 'https://www.thairath.co.th/news/local/2594348': 2, 'https://www.thairath.co.th/': 1, 'https://www.thairath.co.th/tv/watch': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang': 4, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330': 2, 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/704356': 2, 'https://www.thairath.co.th/lottery': 3, 'https://www.thairath.co.th/business/investment/gold/history': 2, 'https://www.thairath.co.th/business/investment/gold': 1, 'https://www.thairath.co.th/business/investment/gold/topic': 1, 'https://www.thairath.co.th/multimedia/gallery': 2, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218': 6, 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208': 6, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860': 6, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851': 6, 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34308': 4, 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225': 4, 'https://www.thairath.co.th/ads/thairathOnline': 1}
d={'https://www.thairath.co.th/news/politic/2611696', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714915', 'https://www.thairath.co.th/spotlight/bualuang-printing', 'https://www.thairath.co.th/lifestyle/tech/2611086', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33851', 'https://www.thairath.co.th/scoop/world', 'https://www.thairath.co.th/multimedia/gallery', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/704356', 'https://www.thairath.co.th/ads/thairathOnline', 'https://www.thairath.co.th/spotlight/covid-19-vaccine', 'https://www.thairath.co.th/news/society/2611708', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/33860', 'https://www.thairath.co.th/tv/program/thairath_news_show', 'https://www.thairath.co.th/news/politic/2611423', 'https://www.thairath.co.th/column/newspaper/wikroh', 'https://www.thairath.co.th/', 'https://www.thairath.co.th/lifestyle/fashion/2611583', 'https://www.thairath.co.th/scoop/culture/2607981', 'https://www.thairath.co.th/lifestyle', 'https://www.thairath.co.th/sport/thaifootball/2611678', 'https://www.thairath.co.th/spotlight', 'https://www.thairath.co.th/news/local/2611523', 'https://www.thairath.co.th/lottery', 'https://www.thairath.co.th/business/investment/gold/history', 'https://www.thairath.co.th/scoop', 'https://www.thairath.co.th/tv/program/buntoeng_thairath', 'https://www.thairath.co.th/tv', 'https://www.thairath.co.th/tv/program', 'https://www.thairath.co.th/podcast', 'https://www.thairath.co.th/scoop/theissue/2611330', 'https://www.thairath.co.th/tv/live', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34308', 'https://www.thairath.co.th/news/local/2610793', 'https://www.thairath.co.th/news/local/2594348', 'https://www.thairath.co.th/scoop/world/2610365', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714921', 'https://www.thairath.co.th/tv/watch', 'https://www.thairath.co.th/podcast/program/Howtolove/episode140', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang', 'https://www.thairath.co.th/horoscope/dailyhoro/2608620', 'https://www.thairath.co.th/column/newspaper/gladai', 'https://www.thairath.co.th/news/politic/2610866', 'https://www.thairath.co.th/scoop/theissue/2610376', 'https://www.thairath.co.th/scoop/interview', 'https://www.thairath.co.th/member/profile/following', 'https://www.thairath.co.th/lifestyle/life/2605256', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714913', 'https://www.thairath.co.th/spotlight/bblfinancialtree/', 'https://www.thairath.co.th/lottery/news/2611380', 'https://www.thairath.co.th/multimedia/gallery/entertain/news/34218', 'https://www.thairath.co.th/lifestyle/life/2611281', 'https://www.thairath.co.th/multimedia/gallery/lifestyle/travel/34225', 'https://www.thairath.co.th/tv/program/talon_khao', 'https://www.thairath.co.th/scoop/theissue/2611335', 'https://www.thairath.co.th/refresh', 'https://www.thairath.co.th/business/investment/gold/topic', 'https://www.thairath.co.th/tags/พ.พาทินี', 'https://www.thairath.co.th/tv/program/buntoeng_thairath/714935', 'https://www.thairath.co.th/tv/watch/news', 'https://www.thairath.co.th/horoscope/horoscope-seer', 'https://www.thairath.co.th/tv/program/khao_chao_hua_khiew', 'https://www.thairath.co.th/tv/program/khao_tieng_thairath', 'https://www.thairath.co.th/tv/program/khoa_yen_thairath', 'https://www.thairath.co.th/news/auto/news/2611385', 'https://www.thairath.co.th/business/market', 'https://www.thairath.co.th/tv/program/khao_sai_khai', 'https://www.thairath.co.th/business/investment/gold', 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033', 'https://www.thairath.co.th/business/market/2608195', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/707330', 'https://www.thairath.co.th/scoop/theissue', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/713101', 'https://www.thairath.co.th/podcast/program/extratime/episode62', 'https://www.thairath.co.th/campaign/vote', 'https://www.thairath.co.th/scoop/interview/2611308', 'https://www.thairath.co.th/column/newspaper/page1scoop', 'https://www.thairath.co.th/lifestyle/life/2610961', 'https://www.thairath.co.th/lifestyle/life', 'https://www.thairath.co.th/scoop/interview/2608225', 'https://www.thairath.co.th/news/foreign/2611565', 'https://www.thairath.co.th/spotlight/onlinelearning', 'https://www.thairath.co.th/column', 'https://www.thairath.co.th/video', 'https://www.thairath.co.th/tv/program/Setthee_Pai_Deang/710265', 'https://www.thairath.co.th/home', 'https://www.thairath.co.th/multimedia/gallery/sport/worldcup/34208', 'https://www.thairath.co.th/lifestyle/fashion', 'https://www.thairath.co.th', 'https://www.thairath.co.th/lifestyle/tech'}
print (len(c))
print(len(d))

for i in b:
    if i not in a :
        print((i))

88
89
https://www.thairath.co.th/tv/program/khao_tieng_thairath/714891
https://www.thairath.co.th/tv/program/khao_tieng_thairath/714880
https://www.thairath.co.th/news
https://www.thairath.co.th/job
https://www.thairath.co.th/aboutus
https://www.thairath.co.th/contact_th
https://www.thairath.co.th/ads
https://www.thairath.co.th/campaign/vote/rule
https://www.thairath.co.th/campaign/vote/prize
https://www.thairath.co.th/tv/schedule
https://www.thairath.co.th/tv/anchor
https://www.thairath.co.th/tv/watch/variety
https://www.thairath.co.th/tv/watch/sport
https://www.thairath.co.th/ads/thairathTV
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714751
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714750
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714745
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714743
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714740
https://www.thairath.co.th/tv/program/khao_chao_hua_khiew/714738
https://ww

In [4]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [12]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  
  if  title_tag == None:
    title_tag = summarize_article[0]
  
  
  
  
  return p_tag, title_tag, keyword

def insert_data_to_db(website, p_tag, title, keyword, website_dict):
  
  for i in keyword:
    conn.execute("INSERT INTO DATA (WEBSITE, BODY, TITLE, KEYWORD, WORD_FREQUENCY ,REF) VALUES (?, ?, ?, ?, ?, ?)", (website, p_tag, title, i, keyword[i],website_dict[website]))
    conn.commit()
    print(f'For website {website}')
    print(f'For the p tags is:{p_tag}') 
    print(f'For the title tag is: {title}')
    print(f'For the keyword is:{i}')
    print(f'Forthe word frequency is:{keyword[i]}')
    print(f'For the ref is:{website_dict[website]}')
 

# Iterate through the websites
conn = sqlite3.connect('scraped_data.db')
conn.execute('''CREATE TABLE DATA
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             WEBSITE STRING NOT NULL,
             BODY TEXT NOT NULL,
             TITLE TEXT NOT NULL,
             KEYWORD TEXT NOT NULL,
             WORD_FREQUENCY INT NOT NULL,
             REF INT NOT NULL);''')
for website in website_dict.keys():
    if 'news' in website:
        p_tag, title, keyword = scrape_tags(website)
        insert_data_to_db(website, p_tag, title, keyword, website_dict)

conn.close()






For website https://www.thairath.co.th/news/local/bangkok/2605735
For the p tags is:ความกดอากาศเย็นจากจีนทําให้วันนี้อุณหภูมิตอนบนของไทยลดลงอีก 1-3 องศาเซลเซียส ขณะมรสุมในภาคใต้แรงขึ้น ทําให้มีฝนฟ้าคะนอง ส่งผลให้คลื่นบริเวณอ่าวไทยสูง 2-3 เมตรความกดอากาศเย็นจากจีนทําให้วันนี้อุณหภูมิตอนบนของไทยลดลงอีก 1-3 องศาเซลเซียส ขณะมรสุมในภาคใต้แรงขึ้น ทําให้มีฝนฟ้าคะนอง ส่งผลให้คลื่นบริเวณอ่าวไทยสูง 2-3 เมตรสําหรับมรสุมตะวันออกเฉียงเหนือที่พัดปกคลุมอ่าวไทย ภาคใต้ และทะเลอันดามันมีกําลังแรงขึ้น ทําให้คลื่นลมบริเวณอ่าวไทยตอนล่างมีกําลังค่อนข้างแรง โดยมีคลื่นสูง 2-3 เมตร บริเวณที่มีฝนฟ้าคะนองคลื่นสูงมากกว่า 3 เมตร ส่วนอ่าวไทยตอนบนมีคลื่นสูงประมาณ 2 เมตร บริเวณที่มีฝนฟ้าคะนองคลื่นสูงมากกว่า 2 เมตร ขอให้ประชาชนที่อาศัยอยู่บริเวณชายฝั่งภาคใต้ฝั่งตะวันออกระวังอันตรายจากคลื่นที่ซัดเข้าหาฝั่ง ส่วนชาวเรือบริเวณอ่าวไทยควรเดินเรือด้วยความระมัดระวังและหลีกเลี่ยงการเดินเรือในบริเวณที่มีฝนฟ้าคะนอง เรือเล็กบริเวณอ่าวไทยตอนล่างควรงดออกจากฝั่งในระยะนี้ไว้ด้วยสําหรับมรสุมตะวันออกเฉียงเหนือที่พัดปกคลุมอ่าวไทย ภาคใต้

AttributeError: 'NoneType' object has no attribute 'items'

In [10]:
keyword = {}
keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
print(keyword)

{}
